In [1]:
import (
        "encoding/json"
        "github.com/FactomProject/ptnet-eventstore/contract"
        "github.com/FactomProject/ptnet-eventstore/ptnet"
)

In [2]:
	c := contract.TicTacToeContract()

In [3]:
[]interface{}{c.ContractID, c.Schema, c.BlockHeight}

[|ContractID| octoe-v1 60221409]

In [4]:
contract.Exists(c.Schema, c.ContractID)

false

In [5]:
event, _ := contract.Create(c, contract.DEPOSITOR_SECRET)
[]interface{}{event.Action, event.InputState, "=>", event.OutputState}

[EXEC [1 1 1 1 1 1 1 1 1 1 1 1 1 1] => [1 1 1 1 1 1 1 1 1 0 1 0 0 0]]

In [6]:
contract.Exists(c.Schema, c.ContractID)

true

In [7]:
contract.IsHalted(c)

false

In [8]:
contract.CanRedeem(c, contract.DEPOSITOR)

false

In [9]:
func commit(action string, key string, payload []byte) (*ptnet.Event, error) {
    return contract.Commit(contract.Command{
		ChainID:    contract.CHAIN_ID,      // chain to write to
		ContractID: contract.CONTRACT_ID,   // uniqueid for this contract execution
		Schema:     ptnet.OctoeV1,          // state machine version
		Action:     action,                 // state machine action
		Amount:     1,                      // triggers input action 'n' times
		Payload:    payload,                // arbitrary data optionally included
		Privkey:    contract.Identity[key], // secret identity used to sign event
		Pubkey:     key,                    // public identity
    })
}

In [10]:
payload, _ := json.Marshal([]string{"hello", "world"})
event1, _ := commit("X11", contract.PLAYERX, payload)

[]interface{}{event1.Action, event1.InputState, "=>", event1.OutputState}

[X11 [1 1 1 1 1 1 1 1 1 0 1 0 0 0] => [1 1 1 1 0 1 1 1 1 1 0 0 0 0]]

In [11]:
// state machine makes players take turns
event2, err := commit("X00", contract.PLAYERX, nil)
[]interface{}{event2.Action, event2.InputState, "=>", err}

[X00 [1 1 1 1 0 1 1 1 1 1 0 0 0 0] => invalid output: -1 offset: 10]

In [12]:
// contract makes players sign each event - here we try to sign with an incorrect key
event3, err := commit("O01", contract.PLAYERX, nil)

[]interface{}{event3.Action, event3.InputState, "=>", err}

[O01 [1 1 1 1 0 1 1 1 1 1 0 0 0 0] => invalid output: -1 offset: 10]

In [13]:
event4, err := commit("O01", contract.PLAYERO, nil)

[]interface{}{event4.Action, event4.InputState, "=>", event4.OutputState}

[O01 [1 1 1 1 0 1 1 1 1 1 0 0 0 0] => [1 0 1 1 0 1 1 1 1 0 1 0 0 0]]

In [14]:
// state machine ensures move is used only once
event5, err := commit("X11", contract.PLAYERX, nil)

[]interface{}{event5.Action, event5.InputState, "=>", err}

[X11 [1 0 1 1 0 1 1 1 1 0 1 0 0 0] => invalid output: -1 offset: 4]

In [15]:
event6, err := commit("X00", contract.PLAYERX, nil)

[]interface{}{event6.Action, event6.InputState, "=>", event6.OutputState}

[X00 [1 0 1 1 0 1 1 1 1 0 1 0 0 0] => [0 0 1 1 0 1 1 1 1 1 0 0 0 0]]

In [16]:
event7, err := commit("O02", contract.PLAYERO, nil)

[]interface{}{event7.Action, event7.InputState, "=>", event7.OutputState}

[O02 [0 0 1 1 0 1 1 1 1 1 0 0 0 0] => [0 0 0 1 0 1 1 1 1 0 1 0 0 0]]

In [17]:
event8, err := commit("X22", contract.PLAYERX, nil)

[]interface{}{event8.Action, event8.InputState, "=>", event8.OutputState}

[X22 [0 0 0 1 0 1 1 1 1 0 1 0 0 0] => [0 0 0 1 0 1 1 1 0 1 0 0 0 0]]

In [18]:
// contract depositor confirms the winner
event9, err := commit("WINX", contract.DEPOSITOR, nil)

[]interface{}{event9.Action, event9.InputState, "=>", event9.OutputState}

[WINX [0 0 0 1 0 1 1 1 0 1 0 0 0 0] => [0 0 0 1 0 1 1 1 0 0 0 0 1 0]]

In [19]:
// contract is only Redeemable after Halting
contract.IsHalted(c)

true

In [20]:
// only the winner can redeem
contract.CanRedeem(c, contract.PLAYERX)

true

In [21]:
contract.CanRedeem(c, contract.PLAYERO)

false